In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_probabilities, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_probabilities: pd.DataFrame
        Датафрейм со столбцами item_id, probability. Сумма probability по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()+документация по np.random.choice
    item_ids = np.array(items_probabilities['item_id'])
    probabilities = np.array(items_probabilities['probability'])

    recs = np.random.choice(item_ids, size=n, replace=False,p=probabilities.tolist())

    return recs.tolist()
    
    

In [5]:
%%time

items_probabilities = (np.log1p(data_train.groupby('item_id')['sales_value'].sum())/np.log1p(data_train.groupby('item_id')['sales_value'].sum()).sum()).reset_index()
items_probabilities.columns=['item_id', 'probability']

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_probabilities, n=5))
print(result.head(2))

   user_id                                             actual  \
0        1  [821867, 834484, 856942, 865456, 889248, 90795...   
1        3  [835476, 851057, 872021, 878302, 879948, 90963...   

                   weighted_random_recommendation  
0     [948225, 15716267, 9676948, 659252, 896749]  
1  [12171326, 949116, 1121410, 1076861, 17104566]  
Wall time: 11.6 s


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [195]:
result = pd.read_csv('data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[15600973, 13511608, 12352235, 8203629, 1006118]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[16223285, 1021420, 1520542, 908299, 13986525]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [201]:
result.shape

(2042, 14)

In [196]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [197]:
#пример bought_list для  пользователя с user_id==1
np.array(result[result['user_id']==1]['actual'])

array(['[  821867   834484   856942   865456   889248   907957   914190   943316\n   951954   954486   958046   962568   969231   971585   979707   986947\n   990656   995242  1004906  1005186  1042083  1050310  1060819  1062002\n  1064441  1069733  1074612  1082185  1131115  1132771  6534544 13876341\n 15971874 17178953   883616   917704   931860   961554  1002032  1031190\n  8090541  8293439  9297615  9527329 15926712  1049998   861272   869465\n   877373   908213   933913   940947   945809   959316   978974  1031697\n  1041796  1048918  1081189  1101422  1115576  1122428  1132231  1132814\n  5577022  8091601  9296986  9677939 10356149 13417048 15741823 15830875]'],
      dtype=object)

In [198]:
#Precision@5 для 1 usera
print('для пользователя с user_id==1 random_recommendation precision_at_5=',precision_at_k(result[result['user_id']==1]['random_recommendation'],result[result['user_id']==1]['actual'],5))
print('для пользователя с user_id==1 popular_recommendation precision_at_5=',precision_at_k(result[result['user_id']==1]['popular_recommendation'],result[result['user_id']==1]['actual'],5))
print('для пользователя с user_id==1 itemitem precision_at_5=',precision_at_k(result[result['user_id']==1]['itemitem'],result[result['user_id']==1]['actual'],5))
print('для пользователя с user_id==1 cosine precision_at_5=',precision_at_k(result[result['user_id']==1]['cosine'],result[result['user_id']==1]['actual'],5))
print('для пользователя с user_id==1 tfidf precision_at_5=',precision_at_k(result[result['user_id']==1]['tfidf'],result[result['user_id']==1]['actual'],5))
print('для пользователя с user_id==1 own_purchases precision_at_5=',precision_at_k(result[result['user_id']==1]['own_purchases'],result[result['user_id']==1]['actual'],5))




для пользователя с user_id==1 random_recommendation precision_at_5= 0.0
для пользователя с user_id==1 popular_recommendation precision_at_5= 0.0
для пользователя с user_id==1 itemitem precision_at_5= 0.0
для пользователя с user_id==1 cosine precision_at_5= 0.0
для пользователя с user_id==1 tfidf precision_at_5= 0.0
для пользователя с user_id==1 own_purchases precision_at_5= 0.0


In [199]:
#вычислим precision_at_k() для всех пользователей и усредним

result['precision_random_recommendation'] = result['user_id'].apply(
    lambda x: precision_at_k(result[result['user_id']==x]['random_recommendation'],result[result['user_id']==x]['actual'], k=5))

result['precision_popular_recommendation'] = result['user_id'].apply(
    lambda x: precision_at_k(result[result['user_id']==x]['popular_recommendation'],result[result['user_id']==x]['actual'], k=5))

result['precision_itemitem'] = result['user_id'].apply(
    lambda x: precision_at_k(result[result['user_id']==x]['itemitem'],result[result['user_id']==x]['actual'], k=5))

result['precision_cosine'] = result['user_id'].apply(
    lambda x: precision_at_k(result[result['user_id']==x]['cosine'],result[result['user_id']==x]['actual'], k=5))

result['precision_tfidf'] = result['user_id'].apply(
    lambda x: precision_at_k(result[result['user_id']==x]['tfidf'],result[result['user_id']==x]['actual'], k=5))

result['precision_own_purchases'] = result['user_id'].apply(
    lambda x: precision_at_k(result[result['user_id']==x]['own_purchases'],result[result['user_id']==x]['actual'], k=5))

print('precision_random_recommendation =',result['precision_random_recommendation'].mean())
print('precision_popular_recommendation =',result['precision_popular_recommendation'].mean())
print('precision_itemitem =',result['precision_itemitem'].mean())
print('precision_cosine =',result['precision_cosine'].mean())
print('precision_tfidf =',result['precision_tfidf'].mean())
print('precision_own_purchases =',result['precision_own_purchases'].mean())


precision_random_recommendation = 0.0
precision_popular_recommendation = 0.0
precision_itemitem = 0.0
precision_cosine = 0.0
precision_tfidf = 0.0
precision_own_purchases = 0.0


In [203]:
# проверка себя -  для пользователя user_id==3 купленные товары==списку из popular_recommendation 
result.loc[result['user_id']==3,'actual']='[6534178, 6533889, 1029743, 6534166, 1082185]'

print('для пользователя с user_id==3 popular_recommendation precision_at_5=',
      precision_at_k(result[result['user_id']==3]['popular_recommendation'],result[result['user_id']==3]['actual'],5))

result['precision_popular_recommendation'] = result['user_id'].apply(
    lambda x: precision_at_k(result[result['user_id']==x]['popular_recommendation'],result[result['user_id']==x]['actual'], k=5))

print('precision_popular_recommendation =',result['precision_popular_recommendation'].mean())

для пользователя с user_id==3 popular_recommendation precision_at_5= 1.0
precision_popular_recommendation = 0.0004897159647404506


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ- {2000, 5000, 10000} товаров. Постройте график зависимости precision@5 от топ-*
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$. Постройте график зависимости precision@5 от $K$ 

In [8]:
# your_code